In [18]:
import cv2
import mediapipe as mp
import autopy
import numpy as np

# Initialize VideoCapture
cap = cv2.VideoCapture(0)

# Initialize mediapipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Get screen size
w_screen, h_screen = autopy.screen.size()

# Initialize previous and current finger positions
prev_finger_pos = [0, 0]
curr_finger_pos = [0, 0]

initHand = mp.solutions.hands
mainHand = initHand.Hands()
draw = mp.solutions.drawing_utils

def handLandmarks(colorImg):
    landmarkList = []
    landmarkPositions = mainHand.process(colorImg)
    landmarkChek = landmarkPositions.multi_hand_landmarks
    if landmarkChek:
        for hand in landmarkChek:
            for index, landmark in enumerate(hand.landmark):  # Change here
                draw.draw_landmarks(frame, hand, initHand.HAND_CONNECTIONS)
                h, w, c = frame.shape
                centerX, centerY = int(landmark.x * w), int(landmark.y * h)
                landmarkList.append([index, centerX, centerY])
    return landmarkList


def fingers(landmarks):
    fingerTips = []
    d = 2
    # Check if middle finger is up
    if landmarks[8][d] < landmarks[7][d]:
        fingerTips.append(1)
    else:
        fingerTips.append(0)
        
    # Check if middle finger is up
    if landmarks[12][d] < landmarks[11][d]:
        fingerTips.append(1)
    else:
        fingerTips.append(0)
        
    # Check if middle finger is up
    if landmarks[16][d] < landmarks[15][d]:
        fingerTips.append(1)
    else:
        fingerTips.append(0)
        
    # Check if middle finger is up
    if landmarks[20][d] < landmarks[19][d]:
        fingerTips.append(1)
    else:
        fingerTips.append(0)
        
    return fingerTips


while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for natural hand movement
    frame = cv2.flip(frame, 1)
    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    lmList = handLandmarks(frame_rgb)
    if len(lmList) != 0:
        finger = fingers(lmList)
    
        # Process the frame to detect hands
        results = hands.process(frame_rgb)
    
        # Draw hand landmarks on the frame
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
                # Get finger landmarks
                finger_landmarks = np.array([[lmk.x * frame.shape[1], lmk.y * frame.shape[0]] for lmk in hand_landmarks.landmark])
    
                # Get index finger position
                index_finger_pos = finger_landmarks[8]
    
                # Convert finger position to screen coordinates
                x, y = int(index_finger_pos[0]), int(index_finger_pos[1])
                x_screen = np.interp(x, [0, frame.shape[1]], [0, w_screen])
                y_screen = np.interp(y, [0, frame.shape[0]], [0, h_screen])
    
                # Smooth out the movement
                curr_finger_pos[0] += (x_screen - curr_finger_pos[0]) / 7
                curr_finger_pos[1] += (y_screen - curr_finger_pos[1]) / 7
    
                # Move the mouse
                autopy.mouse.move(w_screen - curr_finger_pos[0], curr_finger_pos[1])
    
                # Update previous finger position
                prev_finger_pos = curr_finger_pos
                
                if finger[0] == 1 and finger[1] == 1 and finger[2] == 1 and finger[3] == 1:
                    autopy.mouse.click()

    # Display the frame
    cv2.imshow('Hand Tracking', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()